# Deploy to ACI

In [1]:
import os
import json

# from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
from azureml.core.webservice import Webservice, AciWebservice
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Webservice
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.exceptions import WebserviceException

C:\Users\makayser\AppData\Roaming\Python\Python36\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (urllib3 1.25.7 (c:\users\makayser\appdata\local\continuum\anaconda3\envs\nlp\lib\site-packages), Requirement.parse('urllib3<1.25,>=1.21.1'), {'requests'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (urllib3 1.25.7 (c:\users\makayser\appdata\local\continuum\anaconda3\envs\nlp\lib\site-packages), Requirement.parse('urllib3<1.25,>=1.21.1'), {'requests'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto w

## One-time auth

In [2]:
# interactive_auth = InteractiveLoginAuthentication(tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47")

# ws = Workspace.get(name='nlp-ml', 
#                 subscription_id='50324bce-875f-4a7b-9d3c-0e33679f5d72', 
#                 resource_group='nlp',
#                auth=interactive_auth)

## Parameters

In [3]:
language = 'de'
env = 'dev'

#NOTE:
# Change language setting in custom.py before deployment!

In [4]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

nlp-ml
nlp
westeurope
50324bce-875f-4a7b-9d3c-0e33679f5d72


## Upload Model Assets

In [5]:
model = Model.register(workspace=ws,
                       model_name=f'nlp_{language}_{env}',                # Name of the registered model in your workspace.
                       model_path=f'../assets/assets_{language}.zip',  # Local file to upload and register as a model.
                       resource_configuration=ResourceConfiguration(cpu=2, memory_in_gb=2),
                       description='NLP test version',
                       tags={'models': 'classification, ner, qa', 'version': '0.1',
                             'language': language, 'environment': env})

print('Name:', model.name)
print('Version:', model.version)

Registering model nlp_de_dev
Name: nlp_de_dev
Version: 3


## Configure Compute

In [6]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=4, memory_gb=4) #, auth_enabled=True)

In [7]:
environment = Environment('farmenv')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
                                                                                'azureml-defaults',
                                                                                'spacy',
                                                                                'transformers',
                                                                                'farm',
                                                                                'numpy',
                                                                                'azure-storage-blob'
                                                                            ],
                                                                 conda_packages=[
                                                                                'pytorch',
                                                                                'torchvision',
                                                                                'gensim',
                                                                                'numpy',
                                                                                'pandas'
                                                                            ])

In [8]:
os.chdir('../code')

## Deploy

In [9]:
service_name = f'nlp-{language}-{env}'

aci_config = AciWebservice.deploy_configuration(cpu_cores=3, memory_gb=3) #, auth_enabled=True)

inference_config = InferenceConfig(entry_script='infer.py',
                                   source_directory='.',
                                   environment=environment)

# Remove any existing service under the same name.
try:
#     Webservice(ws, service_name).delete()
    # Retrieve existing service.
    service = Webservice(name=service_name, workspace=ws)
    # Update to new model(s).
    service.update(models=[model], inference_config=inference_config)
except WebserviceException:
    service = Model.deploy(ws, service_name, [model], inference_config, deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

Running.....................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [10]:
service.get_keys

<bound method Webservice.get_keys of AciWebservice(workspace=Workspace.create(name='nlp-ml', subscription_id='50324bce-875f-4a7b-9d3c-0e33679f5d72', resource_group='nlp'), name=nlp-de-dev, image_id=None, compute_type=None, state=ACI, scoring_uri=Healthy, tags=http://57012932-730f-429e-9488-ffd07e5e62e7.westeurope.azurecontainer.io/score, properties={})>

In [11]:
service.get_logs()

'/bin/bash: /azureml-envs/azureml_8de03559e2dcd264a4f9a15817343eb6/lib/libtinfo.so.5: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_8de03559e2dcd264a4f9a15817343eb6/lib/libtinfo.so.5: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_8de03559e2dcd264a4f9a15817343eb6/lib/libtinfo.so.5: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_8de03559e2dcd264a4f9a15817343eb6/lib/libtinfo.so.5: no version information available (required by /bin/bash)\nbash: /azureml-envs/azureml_8de03559e2dcd264a4f9a15817343eb6/lib/libtinfo.so.5: no version information available (required by bash)\n2019-12-28T13:22:53,672867654+00:00 - gunicorn/run \n2019-12-28T13:22:53,674547169+00:00 - rsyslog/run \n2019-12-28T13:22:53,674589769+00:00 - iot-server/run \n2019-12-28T13:22:53,691757325+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_8de03559e2dcd264a4f9a15817343eb6/lib/libcrypt

## Test Service

In [12]:
service.run(json.dumps([{"body": "Mein Windows Vista rechner will nicht mehr - ich kriege dauernd fehler meldungen. Ich wollte mir eh einen neuen kaufen, aber ich hab kein Geld. Kann Bill Gates mir helfen?"}]))

[{'task': 1,
  'params': {'label': 'subcat',
   'type': 'classification',
   'language': 'de',
   'prepare': None},
  'result': 'protect'},
 {'task': 3,
  'params': {'type': 'ner', 'language': 'de', 'prepare': None},
  'result': [{'value': 'Windows Vista',
    'start': 5,
    'end': 18,
    'label': 'MISC'},
   {'value': 'mir eh', 'start': 93, 'end': 99, 'label': 'ORG'},
   {'value': 'Bill Gates', 'start': 149, 'end': 159, 'label': 'PER'},
   {'value': 'Windows Vista', 'start': 1, 'end': 3, 'label': 'Product'}]},
 {'task': 4,
  'params': {'type': 'qa', 'language': 'de', 'prepare': None},
  'result': [{'question_clean': 'installation microsoft security essential vista mehrmals versuchen security essential installieren schluß meldung fehler installieren handeln fehler code per online hilfe microsoft trotzen bemühung fehler beheben notebook vista geschenen problem security essential pc windows xp installieren erfahrung helfen',
    'answer_text_clean': 'Bitte etwas mehr Informationen.hast

In [13]:
service.run(json.dumps([{"subject": "My pc won't start:","body": "My pc won't start:"}]))

[{'task': 1,
  'params': {'label': 'subcat',
   'type': 'classification',
   'language': 'de',
   'prepare': None},
  'result': 'xbox'},
 {'task': 3,
  'params': {'type': 'ner', 'language': 'de', 'prepare': None},
  'result': [{'value': "My pc won't",
    'start': 0,
    'end': 11,
    'label': 'MISC'}]},
 {'task': 4,
  'params': {'type': 'qa', 'language': 'de', 'prepare': None},
  'result': [{'question_clean': 'halo master chief collection won t launch i wanted to play halo on pc but when i installed it it won t even launch i would click play the xbox app would minimize like normal i waited for couple minutes and it didn t launch then i reinstalled it and i waited like pns minutes and it wouldn t open',
    'answer_text_clean': 'There has been many people with the same problem. Try starting the game from the start menu instead of the Xbox App. It will be stuck on the splash screen for a few minutes but wait it out and the game should launch fine then. https://news.softpedia.com/news/h